#Lab 06 Code

Carson Dod xbh4ya

In [1]:
#Initial code here copied from final and labs to import file to this particular file
from google.colab import drive
import pandas as pd
import numpy as np
import ast
from collections import Counter
import matplotlib.pyplot as plt
import seaborn as sns


drive.mount('/content/drive', force_remount=True)
file_path = '/content/drive/My Drive/2nd Year/Colab Notebooks/RailAccidents21.csv'

df = pd.read_csv(file_path)
print(df.head())

Mounted at /content/drive
   IYR  IMO RAILROAD     INCDTNO  IYR2  IMO2  RR2 INCDTNO2  IYR3  IMO3  ...  \
0   21   11     SERA  SERA202112   NaN   NaN  NaN      NaN    21    11  ...   
1   21    9       UP   0921NC034   NaN   NaN  NaN      NaN    21     9  ...   
2   21    9      ATK      168291   NaN   NaN  NaN      NaN    21     9  ...   
3   21    3       UP   0321GP007   NaN   NaN  NaN      NaN    21     3  ...   
4   21   10       IC     1086086   NaN   NaN  NaN      NaN    21    10  ...   

  NARR15 RCL   Latitude    Longitud  SIGNAL  MOPERA  ADJUNCT1  ADJUNCT2  \
0    NaN   0  37.865673 -120.472446     2.0     2.0         P       NaN   
1    NaN   0  37.813846 -122.302532     2.0     5.0       NaN       NaN   
2    NaN   0  37.427196 -121.977542     1.0     1.0         Q       NaN   
3    NaN   0  39.798312 -104.998511     2.0     5.0         K       NaN   
4    NaN   0  41.832883  -87.732217     2.0     5.0       NaN       NaN   

   ADJUNCT3             SUBDIV  
0       NaN    

In [2]:
#13
mean_accdmg = round(df['ACCDMG'].mean(), 2)
mean_accdmg

167742.76

In [3]:
#14
max_eqpdmg = df['EQPDMG'].max()
max_eqpdmg

21947116

In [5]:
#15:
highest_eqpdmg_month = df.loc[df['EQPDMG'].idxmax(), 'IMO']

highest_eqpdmg_month
#9 - september

9

In [6]:
#16:
total_passkld = df['PASSKLD'].sum()
total_passkld

3

In [7]:
#17:
total_rrempkld = df['RREMPKLD'].sum()
total_rrempkld

0

In [8]:
#18:
columns_to_check = ['IMO', 'SUBDIV', 'INCDTNO', 'RAILROAD']
no_missing_values = [col for col in columns_to_check if df[col].isnull().sum() == 0]
no_missing_values

['IMO', 'INCDTNO', 'RAILROAD']

In [17]:
#19:
iyr2_null_values = df['IYR2'].isnull().sum()
print(iyr2_null_values)

iyr_null_values = df['IYR'].isnull().sum()
print(iyr_null_values)

correlation_accedmg_eqpdmg = df[['ACCDMG', 'EQPDMG']].corr().iloc[0, 1]
print(correlation_accedmg_eqpdmg)


trnspd_skewness = df['TRNSPD'].skew()
print(trnspd_skewness < 0)



2062
0
0.7527720703944135
False


In [21]:
#20:
total_words_narr1 = df['NARR1'].dropna().str.split().str.len().sum()
total_words_narr1

38931

In [22]:
#21:
first_10_narratives = df['NARR1'].head(10).dropna()
total_capital_letters = first_10_narratives.apply(lambda x: sum(1 for char in x if char.isupper())).sum()
total_capital_letters


650

In [45]:
#22:
import string
from collections import Counter
from nltk.corpus import stopwords
import re
import pandas as pd
import nltk
nltk.download('stopwords')

def clean_text_simple(text):
    text = text.lower()
    tokens = re.findall(r'\b\w+\b', text)
    stop_words = {
        "a", "an", "the", "and", "or", "not", "is", "are", "was", "were",
        "in", "on", "at", "of", "to", "from", "with", "by", "for", "as"
    }
    tokens = [word for word in tokens if word not in stop_words]
    return tokens

df['cleaned_narratives'] = df['NARR1'].dropna().apply(lambda x: clean_text_simple(str(x)))

all_words = [word for sublist in df['cleaned_narratives'] for word in sublist]
word_freq = Counter(all_words)
most_common_words = word_freq.most_common(5)

for word, freq in most_common_words:
    print(word, freq)

track 937
cars 746
train 646
derailed 566
while 484


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [28]:
#23:
derailment_count = df[df['TYPE'] == 1].shape[0]
derailment_count


1220

In [29]:
#24
rainy_weather_count = df[df['WEATHER'] == 3].shape[0]
rainy_weather_count


175

In [30]:
#25:
highest_trnspd_rainy = df[df['WEATHER'] == 3]['TRNSPD'].max()
highest_trnspd_rainy


109

In [31]:
#26:
columns = ['WEATHER', 'VISIBLTY', 'TRNSPD', 'TONS', 'TEMP', 'TYPE']
total_nulls = df[columns].isnull().sum().sum()
total_nulls


2

In [32]:
#27:
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

columns = ['WEATHER', 'VISIBLTY', 'TRNSPD', 'TONS', 'TEMP']
X = df[columns].dropna()
y = (df.loc[X.index, 'TYPE'] == 1).astype(int)

X_train, X_test = X.iloc[:2000], X.iloc[2000:]
y_train, y_test = y.iloc[:2000], y.iloc[2000:]

model = KNeighborsClassifier(n_neighbors=3)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred) * 100

print(accuracy)


58.80281690140845


In [33]:
#28:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import precision_score

columns = ['WEATHER', 'VISIBLTY', 'TRNSPD', 'TONS', 'TEMP']
X = df[columns].dropna()
y = (df.loc[X.index, 'TYPE'] == 1).astype(int)

X_train, X_test = X.iloc[:2000], X.iloc[2000:]
y_train, y_test = y.iloc[:2000], y.iloc[2000:]

rf_model = RandomForestClassifier(max_depth=3, random_state=0)
rf_model.fit(X_train, y_train)

y_pred = rf_model.predict(X_test)

precision = precision_score(y_test, y_pred) * 100

print(precision)


70.2020202020202


In [35]:
#29:
feature_importances = rf_model.feature_importances_
features = columns
most_important_feature = features[feature_importances.argmax()]
most_important_feature


'TONS'